티스토리 블로그 : https://today-1.tistory.com/41

In [3]:
# data load
import pandas as pd
import numpy as np
from pandas.tseries.offsets import Day, Hour, Minute, Second

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

print('train shape: ', train.shape)
print('test shape: ', test.shape)
train.head()

train shape:  (10886, 12)
test shape:  (6493, 9)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
# prophet 라이브러리 임포트
# %pip install pystan==2.19.1.1
# %pip install prophet

from prophet import Prophet

# Prophet 시각화 도구
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import add_changepoints_to_plot

In [5]:
# target 데이터를 ds, y로 전처리
y_train_prophet_df = train[['datetime','count']]
y_train_prophet_df.columns = ['ds', 'y']
y_train_prophet_df.head()

,ds,y
0,2011-01-01 00:00:00,16
1,2011-01-01 01:00:00,40
2,2011-01-01 02:00:00,32
3,2011-01-01 03:00:00,13
4,2011-01-01 04:00:00,1


In [6]:
# ds, y를 제외한 x 변수들을 따로 df 처리
x_col = [i for i in train.columns if i not in ['datetime','count']]
x_train_prophet_df = train[x_col]
x_train_prophet_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered
0,1,0,0,1,9.84,14.395,81,0.0,3,13
1,1,0,0,1,9.02,13.635,80,0.0,8,32
2,1,0,0,1,9.02,13.635,80,0.0,5,27
3,1,0,0,1,9.84,14.395,75,0.0,3,10
4,1,0,0,1,9.84,14.395,75,0.0,0,1


In [7]:
# propeht 학습 시에는 꼭 df, y, x 변수 순서대로 쌓인 df를 밀어넣어야 한다.
yx_train_prophet_df = pd.concat([y_train_prophet_df, x_train_prophet_df], axis=1)

In [ ]:
# prophet 모델링 기본
# Prophet Model에 hyper parameter 설명은 아래와 같음
# 크게 5가지로 1) 선형/비선형 선택, 2) 추세, 3) 계절성, 4) 휴일, 5) 기타 등으로 구분됨
# ARIMA, SARIMA 등 통계 기반 모델의 hyper parameter 와는 달리 기술적으로 참고해서 입력할 수 있는 para가 거의 없음
# 실제 현업을 잘 이해하고 있는 전문가들이 예측하고자 하는 문제와 관련된 이벤트, 민감도 외 기타 등등 핸들링이 용이함

fit_default_prophet = Prophet(
    # linear, nonlinear
    growth='linear',

    # Trend (추세)
    changepoints=None,              # CP가 발생하는 시점들의 list
    n_changepoints=25,              # CP의 수
    changepoint_range=0.8,          # CP의 기존 데이터 수 대비 최대 비율
    changepoint_prior_scale=0.05,   # CP 추정 민감도로 높을수록 민감

    
)